In [ ]:
import openpathsampling as ops
import openpathsampling.engines.toy as toys
from potentials import DoublewellPotential, ZPotential
from toyPlot import ToyPlot
import numpy as np
import random
from datetime import datetime
import pickle
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2

In [ ]:
PES_type = "ZP"
assert PES_type == "DW" or PES_type == "ZP", \
    "PES_type must be chosen to match either 'DW' or 'ZP'"
simulation_length = 1000000
additional_dim_generator = "HO"
assert additional_dim_generator == "HO" or additional_dim_generator == "UR", \
    "additional_dim_generator must be chosen to match either 'HO' or 'UR'"
additional_dimensions = 8

In [ ]:
def prepare_doublewell_potential():
    pes = DoublewellPotential()
    plot = ToyPlot((-1.1, 1.1), (-1.1, 1.1))
    plot.contour_range = np.arange(-1.0, 1.5, 0.1)
    plot.add_pes(pes)
    return plot, pes

def prepare_Z_potential():
    pes = ZPotential()
    plot = ToyPlot((-20.1, 20.1), (-20.1, 20.1))
    plot.contour_range = np.arange(-1.5, 7.5, 0.5)
    plot.add_pes(pes)
    return plot, pes

if PES_type == "DW":
    plot, pes = prepare_doublewell_potential()
elif PES_type == "ZP":
    plot, pes = prepare_Z_potential()

plot.plot()
pass

In [ ]:
topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)

if PES_type == "DW":
    integrator_params = {"dt": 0.02, "temperature": 0.1, "gamma": 2.5}
elif PES_type == "ZP":
    integrator_params = {"dt": 0.2, "temperature": 1.0, "gamma": 1.0}

toy_engine = toys.Engine(
    {'integ': toys.LangevinBAOABIntegrator(**integrator_params),
     'n_frames_max': 50000, 'n_steps_per_frame': 1}, topology)
toy_engine.current_snapshot = toys.Snapshot(
    coordinates=np.array([[0.0, 0.0]]),
    velocities=np.array([[0.0, 0.0]]),
    engine=toy_engine)

In [ ]:
def circle(snapshot, center):
    import math
    return math.sqrt(
        (snapshot.xyz[0][0] - center[0])**2
        + (snapshot.xyz[0][1] - center[1])**2)

def disk(snapshot, center):
    import math
    return math.sqrt(
        (snapshot.xyz[0][0]-center[0])**2/16 
        + (snapshot.xyz[0][1]-center[1])**2)

if PES_type == "DW":
    opA = ops.CoordinateFunctionCV(
        name="opA", f=circle, center=[-0.5, 0.0])
    stateA = ops.CVDefinedVolume(opA, 0.0, 0.2).named("A")
    opB = ops.CoordinateFunctionCV(
        name="opB", f=circle, center=[0.5, 0.0])
    stateB = ops.CVDefinedVolume(opB, 0.0, 0.2).named("B")
elif PES_type == "ZP":
    opA = ops.CoordinateFunctionCV(
        name="opA", f=disk, center=[-7.2, -5.1])
    stateA = ops.CVDefinedVolume(opA, 0.0, 0.25).named("A")
    opB = ops.CoordinateFunctionCV(
        name="opB", f=disk, center=[7.2, 5.1])
    stateB = ops.CVDefinedVolume(opB, 0.0, 0.25).named("B")

In [ ]:
long_trajectory = \
    toy_engine.generate_n_frames(n_frames = simulation_length)

In [ ]:
Ens_1InA = ops.IntersectionEnsemble(
    ops.AllInXEnsemble(stateA), ops.LengthEnsemble(1))
Ens_1InB = ops.IntersectionEnsemble(
    ops.AllInXEnsemble(stateB), ops.LengthEnsemble(1))
Ens_NotInAB = ops.IntersectionEnsemble(
    ops.AllOutXEnsemble(stateA), ops.AllOutXEnsemble(stateB))

print("Starting: AB")
AB_network = ops.SequentialEnsemble((Ens_1InA,Ens_NotInAB,Ens_1InB))
subtrajectories_AB = AB_network.split(long_trajectory)
print(len(subtrajectories_AB))

print("Starting: BA")
BA_network = ops.SequentialEnsemble((Ens_1InB,Ens_NotInAB,Ens_1InA))
subtrajectories_BA = BA_network.split(long_trajectory)
print(len(subtrajectories_BA))

print("Starting: AA")
AA_network = ops.SequentialEnsemble((Ens_1InA,Ens_NotInAB,Ens_1InA))
subtrajectories_AA = AA_network.split(long_trajectory)
print(len(subtrajectories_AA))

print("Starting: BB")
BB_network = ops.SequentialEnsemble((Ens_1InB,Ens_NotInAB,Ens_1InB))
subtrajectories_BB = BB_network.split(long_trajectory)
print(len(subtrajectories_BB))

In [ ]:
plot.plot(subtrajectories_AB)
plot.plot(subtrajectories_BA)
plot.plot(subtrajectories_AA)
plot.plot(subtrajectories_BB)
pass

In [ ]:
steps_in = {}
steps_in["AA"] = sum([len(trajectory) for trajectory in subtrajectories_AA])
steps_in["AB"] = sum([len(trajectory) for trajectory in subtrajectories_AB])
steps_in["BA"] = sum([len(trajectory) for trajectory in subtrajectories_BA])
steps_in["BB"] = sum([len(trajectory) for trajectory in subtrajectories_BB])
steps_in["any path"] = sum([value for key, value in steps_in.items()])
percentage_out_of_states = 100 * steps_in["any path"] / simulation_length
print(f"Simulation of {simulation_length} steps was decomposed into:\n"
      + f"{steps_in['any path']} steps ({percentage_out_of_states}%) spend"
      + " in total outside of the stable states.")
print(f"{steps_in['AA']} steps spend in AA trajectories.")
print(f"{steps_in['AB']} steps spend in AB trajectories.")
print(f"{steps_in['BA']} steps spend in BA trajectories.")
print(f"{steps_in['BB']} steps spend in BB trajectories.")

In [ ]:
total_array_AA = np.array([np.array([xy_position[0] \
    for xy_position in trajectory.xyz]) \
    for trajectory in subtrajectories_AA])
total_array_AB = np.array([np.array([xy_position[0] \
    for xy_position in trajectory.xyz]) \
    for trajectory in subtrajectories_AB])
total_array_BA = np.array([np.array([xy_position[0] \
    for xy_position in trajectory.xyz]) \
    for trajectory in subtrajectories_BA])
total_array_BB = np.array([np.array([xy_position[0] \
    for xy_position in trajectory.xyz]) \
    for trajectory in subtrajectories_BB])

trajectory_list = np.array(
    [trajectory for trajectory in total_array_AA]
    + [trajectory for trajectory in total_array_AB]
    + [trajectory for trajectory in total_array_BA]
    + [trajectory for trajectory in total_array_BB])

trajectory_label_list = np.array(
    ["AA" for trajectory in total_array_AA]
    + ["AB" for trajectory in total_array_AB]
    + ["BA" for trajectory in total_array_BA]
    + ["BB" for trajectory in total_array_BB])

print(trajectory_list.shape)
print(trajectory_label_list.shape)

In [ ]:
def generate_additional_dimensions_via_harmonic_oscillator(
        additional_dims, steps_in_any_path):
    pes = toys.HarmonicOscillator(
        additional_dims * [1.0], additional_dims * [1.0], additional_dims * [0.0])
    topology = toys.Topology(
        n_spatial=additional_dims, masses=additional_dims *[1.0], pes=pes)
    toy_engine = toys.Engine(
        {'integ': toys.LangevinBAOABIntegrator(**integrator_params),
         'n_frames_max': 50000, 'n_steps_per_frame': 1}, topology)
    template = toys.Snapshot(
        coordinates=np.array([additional_dims * [0.0]]),
        velocities=np.array([additional_dims * [0.0]]),
        engine=toy_engine)
    toy_engine.current_snapshot = template
    trajectory = toy_engine.generate_n_frames(n_frames = steps_in_any_path)
    return trajectory.xyz[:,0]

def generate_additional_dimensions_via_uniform_random(
        additional_dims, steps_in_any_path):
    return np.array([[(random.random()-0.5)*2 
                    for i in range(additional_dims)]
                    for j in range(steps_in_any_path)])

In [ ]:
if additional_dim_generator == "HO":
    additional_dimension_entries = \
        generate_additional_dimensions_via_harmonic_oscillator(
            additional_dimensions, steps_in["any path"])
elif additional_dim_generator == "UR":
    additional_dimension_entries = \
        generate_additional_dimensions_via_uniform_random(
            additional_dimensions, steps_in["any path"])

In [ ]:
def extend_path_trajectories_with_additional_dimensions(
        trajectory_list, additional_dimension_entries):
    full_trajectory_list = []
    traj_start = 0
    for trajectory in trajectory_list:
        traj_end = traj_start + len(trajectory)
        add_trajectory = additional_dimension_entries[traj_start:traj_end]
        full_trajectory_list.append(
            np.concatenate((trajectory, add_trajectory),axis =1))
        traj_start = traj_end
    return np.array(full_trajectory_list)

extended_trajectory_list = extend_path_trajectories_with_additional_dimensions(
        trajectory_list, additional_dimension_entries)

In [ ]:
time_stamp = str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
pickle.dump(
    extended_trajectory_list,
    open(f"{PES_type}_{time_stamp}_paths.p", "wb"))
pickle.dump(
    trajectory_label_list,
    open(f"{PES_type}_{time_stamp}_labels.p", "wb"))